This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [110]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [111]:
# First, import the relevant modules
import requests


In [112]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-02&end_date=2017-01-02&api_key=' + API_KEY
r = requests.get(url)

In [113]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data = r.json()
print(type(data))
print(data)

<class 'dict'>
{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2018-10-30T22:30:47.066Z', 'newest_available_date': '2018-10-30', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2017-01-02', 'end_date': '2017-01-02', 'data': [['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [114]:
# same url as before with adjusted start_date and end_date to get the whole year
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY
r = requests.get(url)

In [115]:
json_dict = r.json()

# Extract the column_names as a list
column_names = json_dict['dataset']['column_names']

# Extract the data as a list of lists
data = json_dict['dataset']['data']

# The first day is 01/02 because 01/01 is a holiday
# The last day is 12/29 because 12/30 and 12/31 are Saturday and Sunday
print(data[0])
print(data[-1])

['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]
['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None]


In [116]:
# This function returns a list of values for one column
def get_col_values(col_name):
    i = column_names.index(col_name)
    col = [day[i] for day in data]
    return col

In [117]:
# Get a list of the opening prices for every day
o_col = get_col_values('Open')

# Find highest and lowest price (min() and max() return error because there are None values)
low = high = o_col[0]
for val in o_col:
    if val is not None: # ignore None values
        if val < low:
            low = val
        if val > high:
            high = val

print('Highest opening price in 2017 = ',high)
print('Lowest opening price in 2017 = ',low)

Highest opening price in 2017 =  53.11
Lowest opening price in 2017 =  34.0


In [118]:
# Get a list of the highest prices and a list of the lowest prices for every day
l_col = get_col_values('Low')
h_col = get_col_values('High')

# Calculate the difference between highest and lowest prices for every day and keep the largest
change = h_col[0] - l_col[0]
for l_val, h_val in zip(l_col, h_col):
    if (l_val is not None) & (h_val is not None):
        if h_val - l_val > change:
            change = h_val - l_val

print('Largest change in any one day = ',round(change,2))

Largest change in any one day =  2.81


In [119]:
# Get a list of the closing price for every day
c_col = get_col_values('Close')

# Calculate the difference with the previous day closing price(yesterday) and keep the largest
yesterday = c_col[0]
change = 0
for val in c_col:
    if val is not None:
        if abs(val-yesterday) > abs(change):
            change = val-yesterday
        yesterday = val

print('Largest change between any two consecutive days = ',round(change,2))

Largest change between any two consecutive days =  2.56


In [120]:
# Get a list of the traded volumes for every day
tv_col = get_col_values('Traded Volume')

# Calculate the average
average = sum(tv_col)/len(tv_col)
print('Average daily trading volume in 2017 = ',round(average,1))

Average daily trading volume in 2017 =  89124.3


In [121]:
# Sort the list of traded volumes
tv_col_s = sorted(tv_col)

# Find median value
size = len(tv_col)
if size%2 == 1 :
    median = tv_col_s[size//2]
else :
    median = (tv_col_s[size//2] + tv_col_s[size//2+1])/2

print('Median daily trading volume in 2017 = ',round(median,1))

Median daily trading volume in 2017 =  76286.0
